Remember to install:

pip install beautifulsoup4
pip install -qU "langchain-chroma>=0.1.2"



### API keys  ###

In [3]:
import getpass
import os


# (optional) LangSmith to inspect inside your chain or agent.
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "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"


In [4]:
import os
from dotenv import load_dotenv, find_dotenv
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_together import TogetherEmbeddings
from langchain_together import ChatTogether

load_dotenv()

# client = Together(api_key = os.getenv("TOGETHER_API_KEY"))

#load model
model = ChatTogether(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
    temperature=0,
    max_tokens=320,
    # top_k=50,
    together_api_key= os.getenv("TOGETHER_API_KEY")
)


### Retriever ###
 - Create index using HSE website 

In [ ]:
# Documents
question = "How do I reduce exposure to UV light"
document = "Reduce UV light by applying sun screen with a high SPF rating."

In [27]:
from langchain_together.embeddings import TogetherEmbeddings
embd = TogetherEmbeddings()

query_result = embd.embed_query(question)
document_result = embd.embed_query(document)

len(query_result)

768

In [28]:
import numpy as np

def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

similarity = cosine_similarity(query_result, document_result)
print("Cosine Similarity:", similarity)

Cosine Similarity: 0.5388953951356528


In [39]:
#### INDEXING ####

# Load blog
import bs4
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader


urls = [
    "https://www2.hse.ie/conditions/skin-cancer-melanoma/",
    "https://www2.hse.ie/conditions/non-melanoma-skin-cancer/",
    "https://www.cancer.ie/cancer-information-and-support/cancer-types/skin-cancer/keep-your-children-safe-in-the-sun",
    ]

docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, chunk_overlap=50
)

doc_splits = text_splitter.split_documents(docs_list)
print(doc_splits)
print(docs)

[Document(metadata={'source': 'https://www2.hse.ie/conditions/skin-cancer-melanoma/', 'title': '\n            \n                Skin cancer (melanoma) overview\n            \n            \n                \n                    - HSE.ie\n                \n            \n        ', 'description': 'Melanoma skin cancer starts in cells in skin called melanocytes. It can spread to other organs in the body. Find out about melanoma', 'language': 'en'}, page_content='Skin cancer (melanoma) overview\n            \n            \n                \n                    - HSE.ie\n                \n            \n        \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to main content\n\n\n\n\n\n\n\n\nMenu\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nToggle search\n\n\n\n\n\n\n\nSearch the website\n\n\n\n\n\n\nSearch\n\n\n\n\n\nRecommended links\n\n\n\n\n\nFind a pharmacy giving flu and COVID-19 vaccines\n\n\nFind a civil registration service\n\n\

In [36]:
# Index
from langchain_together import TogetherEmbeddings
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(documents=doc_splits, 
                                    embedding = TogetherEmbeddings(model="togethercomputer/m2-bert-80M-8k-retrieval")
                                    )

retriever = vectorstore.as_retriever()

In [38]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

prompt = hub.pull("rlm/rag-prompt")

# Chain
rag_chain = (
    {"context": retriever , "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

output = rag_chain.invoke("What are the most common signs of melanoma?")
print(output)

The most common signs of melanoma include a new or changing mole, a raised lump on the skin that might be black or red, and a flat and brown mole that develops slowly over years. These signs can appear on areas of the skin that are often exposed to the sun, such as the face, scalp, hands, and feet. If you notice any of these changes, it's essential to contact your GP for further evaluation.


### end of the top part is working ###

 - Why doesn't Chroma not work?
   - Works for FAISS, asks questions with correct input.
 - Problem 1,463 tokens uses 1,375 prompt tokens and 88 completion tokens.

In [10]:
import bs4
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_together.embeddings import TogetherEmbeddings

# knowledge
# urls = [
#     "https://www2.hse.ie/conditions/skin-cancer-melanoma/",
#     "https://www2.hse.ie/conditions/non-melanoma-skin-cancer/",
# ]

loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# Embed
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=TogetherEmbeddings(model="togethercomputer/m2-bert-80M-8k-retrieval")
                )   
retriever = vectorstore.as_retriever()


KeyboardInterrupt: 

Bottom indexer works, chroma.from_documents keeps giving me trouble? Also makes use of vectors formed from texts.

In [40]:
from operator import itemgetter
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_together import TogetherEmbeddings
from langchain_community.llms import Together

vectorstore = FAISS.from_texts(
    ["There are 2 common types of skin cancer - melanoma skin cancer and non-melanoma skin cancer.",
"Melanoma skin cancer is much less common than non-melanoma skin cancer. But it has become more common in Ireland in recent years. About 1,100 people are diagnosed with melanoma skin cancer each year.",
"It is more common in people over 50, but you can get it at any age.",
"Melanoma skin cancer starts in cells in skin called melanocytes. It can spread to other organs in the body.",],
     TogetherEmbeddings(model="togethercomputer/m2-bert-80M-8k-retrieval")
)

retriever = vectorstore.as_retriever()

print(retriever)

tags=['FAISS', 'TogetherEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002143D683F10> search_kwargs={}


Simple response example:

In [41]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

# Chain
rag_chain = (
    {"context": retriever , "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

output = rag_chain.invoke("What are the two common types of skin cancer? What is Non Melanoma")
print(output)

The two common types of skin cancer are melanoma skin cancer and non-melanoma skin cancer. Non-melanoma skin cancer is more common than melanoma skin cancer. It is less severe but still requires medical attention.


### CRAGS ###

- use the grader and see how much tokens this will bring the response up by?
- what is the data given, use the lang respose x

- grading current docs, if none are relevant, perform a websearch.

In [15]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field


# Data model
class GradeDocuments(BaseModel):
    """Binary score for relevance check on retrieved documents."""

    binary_score: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
    )

structured_llm_grader = model.with_structured_output(GradeDocuments)

# Prompt
system = """You are a grader assessing relevance of a retrieved document to a user question. \n 
    If the document contains keyword(s) or semantic meaning related to the question, grade it as relevant. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""
grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)

retrieval_grader = grade_prompt | structured_llm_grader
question = "agent memory"
docs = retriever.get_relevant_documents(question)
doc_txt = docs[1].page_content
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))


c:\Users\solom\AppData\Local\Programs\Python\Python311\Lib\site-packages\IPython\core\interactiveshell.py:3550: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\solom\AppData\Local\Temp\ipykernel_18360\3057287797.py:28: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(quest

binary_score='no'
